In [3]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
collection_all_faces = client["faceDB"]["allFaceRecords"]

# Count documents in the collection
count = collection_all_faces.count_documents({})
print("Number of records in the collection: %d" % count)

Number of records in the collection: 605949


In [10]:
import os
import glob

def delete_images(folder):
    # Get all image files in the folder and its subfolders
    image_files = glob.glob(os.path.join(folder, '**', '*.*'), recursive=True)
    print(len(image_files))
    
    for image_file in image_files:
        # Get the filename without extension
        if not image_file.endswith('.jpg'):continue
        filename = os.path.basename(image_file)
        
        filename = (filename.split('_')[0])
        
        # Check if the filename is a number and not divisible by 30
        if filename.isdigit() and int(filename) % 30 != 0:
            try:
                os.remove(image_file)
                print("Deleted %s" % image_file)
            except FileNotFoundError:
                print("File not found: %s" % image_file)

# Use the function
delete_images('/home/dev/face_retrieval/cropped_faces')

606882


In [ ]:
from face_encoder_from_video_path import ProcessVideo
from pytube import YouTube
from pytube.exceptions import AgeRestrictedError
import pandas as pd
from pytube import YouTube
import unicodedata
import os
import gc
import time
from pymongo import MongoClient
from memory_profiler import profile
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
from pytube.exceptions import AgeRestrictedError
from http.client import IncompleteRead
from pytube.exceptions import VideoUnavailable

In [ ]:
def download_video_from_youtube(link, path, retries=3):
    for _ in range(retries):
        try:
            yt = YouTube(link)
            video = yt.streams.get_highest_resolution()
            # save the name of the video as unicode and unsigned
            video_name = unicodedata.normalize('NFKD', video.title).encode('ascii', 'ignore').decode('utf-8')
            video_name = video_name.replace(' ', '_')
            video_name = video_name.replace('\"', '_')
            video_name = video_name.replace('\'', '_')
            video_name = video_name.replace('/', '_')
            video_name = video_name.replace(',', '_')
            # download the video
            video.download(path, filename=video_name+'.mp4')
            return os.path.join(path, video_name+'.mp4')
        except AgeRestrictedError:
            print(f"{bcolors.WARNING}Skipping age-restricted video: {link}")
            return None
        except IncompleteRead:
            print('Incomplete read, retrying download...')
        except VideoUnavailable:
            print(f"Video {link} is unavailable")
            return None
    print('Failed to download video after multiple attempts')
    return None

In [ ]:
link = 'https://www.youtube.com/watch?v=RXiS2AsXjpg'
path = '/home/dev/face_retrieval/videos'
download_video_from_youtube(link, path, retries=3)